In [1]:
import pyglet
import geoplotlib
from geoplotlib.colors import create_set_cmap
from sklearn.cluster import KMeans
import geoplotlib
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.utils import BoundingBox
import pandas as pd
import numpy as np

In [2]:
file = 'new_2013_mls.csv'

In [3]:
df = pd.read_csv(file, encoding = "ISO-8859-1")

In [4]:
df.dtypes

Unnamed: 0            int64
Zip                 float64
Sqft Total           object
List Price           object
Sold/Lease$/SqFt     object
Sold/Lease Price     object
Sold/Lease Date      object
dtype: object

In [6]:
# The last row (11583) is filled with NaN values and I am displaying this and will be cleaning
df.tail()

,Unnamed: 0,Zip,Sqft Total,List Price,Sold/Lease$/SqFt,Sold/Lease Price,Sold/Lease Date
11579,11579,78735.0,"5,995","$2,275,000",$354.46,"$2,125,000",5/28/2013
11580,11580,78735.0,"7,940","$2,499,000",$277.08,"$2,200,000",6/14/2013
11581,11581,78736.0,"7,575","$2,500,000",$305.20,"$2,311,857",10/4/2013
11582,11582,78735.0,"7,346","$3,250,000",$422.00,"$3,100,000",9/10/2013
11583,11583,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Using .rename(columns={}) in order to rename columns
renamed_df = df.rename(columns={"Zip":"Zip Code", "Sqft Total":"Square Footage", "Sold/Lease$/SqFt":"Sold Price Per Foot","Sold/Lease Date":"Sold Date","Sold/Lease Price":"Sold Price"})
del renamed_df['Unnamed: 0']
renamed_df = renamed_df.drop(renamed_df.index[-1])
renamed_df = renamed_df.fillna(0)

In [8]:
for idx, row in renamed_df.iterrows():
    renamed_df.set_value(idx,'Sold Price', float(str(row['Sold Price']).replace(',','').replace('$','').strip()))
    renamed_df.set_value(idx,'Zip Code', float(str(row['Zip Code']).replace(',','').replace('$','').strip()))
    renamed_df.set_value(idx, 'Square Footage', float(str(row['Square Footage']).replace(',','').replace('$','').strip()))
    renamed_df.set_value(idx,'List Price', float(str(row['List Price']).replace(',','').replace('$','').strip()))
    renamed_df.set_value(idx,'Sold Price Per Foot', float(str(row['Sold Price Per Foot']).replace(',','').replace('$','').strip()))

renamed_df.tail()

,Zip Code,Square Footage,List Price,Sold Price Per Foot,Sold Price,Sold Date
11578,78735.0,4822,1.875e+06,368.1,1.775e+06,6/11/2013
11579,78735.0,5995,2.275e+06,354.46,2.125e+06,5/28/2013
11580,78735.0,7940,2.499e+06,277.08,2.2e+06,6/14/2013
11581,78736.0,7575,2.5e+06,305.2,2.31186e+06,10/4/2013
11582,78735.0,7346,3.25e+06,422,3.1e+06,9/10/2013


In [9]:
renamed_df.dtypes

Zip Code               float64
Square Footage          object
List Price              object
Sold Price Per Foot     object
Sold Price              object
Sold Date               object
dtype: object

In [10]:
renamed_df['Sold Price'] = renamed_df['Sold Price'].astype(int)
renamed_df['Zip Code'] = renamed_df['Zip Code'].astype(int)
renamed_df['Square Footage'] = renamed_df['Square Footage'].astype(int)
renamed_df['Sold Price Per Foot'] = renamed_df['Sold Price Per Foot'].astype(float)
renamed_df['List Price'] = renamed_df['List Price'].astype(int)

In [11]:
renamed_df.dtypes

Zip Code                 int32
Square Footage           int32
List Price               int32
Sold Price Per Foot    float64
Sold Price               int32
Sold Date               object
dtype: object

In [12]:
renamed_df.describe()

,Zip Code,Square Footage,List Price,Sold Price Per Foot,Sold Price
count,11583.000000,11583.000000,1.158300e+04,11583.000000,1.158300e+04
mean,78736.860485,2278.491410,3.818551e+05,160.048443,3.730506e+05
std,15.261731,1072.650173,3.539743e+05,78.412336,3.284826e+05
min,78613.000000,0.000000,3.000000e+04,0.000000,4.000000e+04
25%,78728.000000,1513.000000,1.999000e+05,111.960000,1.972500e+05
50%,78738.000000,2046.000000,2.890000e+05,136.660000,2.854900e+05
75%,78748.000000,2821.500000,4.350000e+05,185.700000,4.290000e+05
max,78759.000000,12878.000000,6.950000e+06,2358.330000,5.860000e+06


In [13]:
renamed_df.head()

,Zip Code,Square Footage,List Price,Sold Price Per Foot,Sold Price,Sold Date
0,78759,2126,379888,177.80,378000,11/22/2013
1,78731,2372,399900,160.20,380000,9/12/2013
2,78731,1662,396000,228.64,380000,7/3/2013
3,78731,2104,449000,182.98,385000,9/16/2013
4,78759,1877,385000,205.11,385000,7/24/2013


In [13]:
# new_2013_df = renamed_df
# new_2013_df.to_csv("clean_2013_mls.csv")